### Objective

At a high level, these strategies all involve providing the model clear and specific instructions for what output it should produce. In this lab, you will:

- Define the output format & specify constraints
- Assign a persona or role
- Include examples
- Experiment with parameter values
- Utilize fallback responses
- Add contextual information
- Structure prompts with prefixes or tags
- Use system instructions
- Break down complex tasks
- Demonstrate the chain-of-thought
- Implement prompt iteration strategies to improve your prompts version by version

### Task 1. Initialize Vertex AI in a Colab Enterprise notebook

Upgrade the Vertex AI SDK & Restart the Kernel

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

# note: Select Runtime > Restart Session > select Yes to restart the runtime.

In [ ]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

### Import packages

In [ ]:
PROJECT_ID = "qwiklabs-gcp-03-f2605c9303f1"  
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

### Task 2. Load a generative model

In [ ]:
model = GenerativeModel("gemini-2.0-flash-001")

### Task 3. Define the output format & specify constraints

In [ ]:
# store a transcript from a customer's order at a fast food restaurant.

transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [ ]:
# Run the following prompt that attempts to understand a customer's order from a conversation in JSON format, but in an unspecific way.

response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

In [ ]:
# Now run a version of this prompt with more specific instructions about exactly how you would like your output structured. Notice how the JSON output now reflects the key pieces of information you are interested in to understand the user's order:

response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

### Task 4. Assign a persona or role

In [ ]:
# Try an example within the context of a chat. To start, create a chat session with your Gemini model:
chat = model.start_chat()

In [ ]:
# Ask for a response without a persona specified:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

In [ ]:
# Now run a version of this prompt with a role specified. Notice how this output might be more personal and appealing for some users.
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

### Task 5. Include examples

In [ ]:
# Run the code below in a new cell to see how it interprets the Customer Message and explains its response
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

### Task 6. Experiment with parameter values

In [ ]:
# Run the code from the following cell a few times. The temperature and top_p parameters which lead to variety in responses are set to low values, so the output should be the same, or very close to the same, each time.
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

In [ ]:
# Now run this version of the code a few times. You'll see that the higher temperature and top_p parameter values now lead to more varied results. Some of the results, however, may be a little too random and not make very much sense. If you want variety in your responses, you'll need to experiment with parameters to determine the right balance of creativity and reliability.
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)